<a href="https://colab.research.google.com/github/MartinekV/DL-for-bio-course/blob/master/02_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries setup

In [ ]:
!pip install -q torchmetrics

## Data preparation and exploration

In [ ]:
import torchvision.datasets as dsets
import torchvision.transforms as transforms

train_dataset = dsets.MNIST(root = './data', train = True, transform = transforms.ToTensor(), download = True)

Pytorch dataset implements two methods


```
__len__ #length of the dataset
__getitem__ #access to a single datapoint
```



In [ ]:
print(train_dataset.__len__())

In [ ]:
counts = {num:0 for num in range(10)}
for x,y in train_dataset:
  counts[y]+=1

counts

In [5]:
sample_index = 123 #from 0 to 59999
sample_X, sample_y = train_dataset.__getitem__(sample_index)

In [ ]:
to_image = transforms.ToPILImage()
resize = transforms.Resize((100,100))
print(sample_y)
resize(to_image(sample_X))


In [ ]:
sample_X

In [ ]:
import torch
#Check if the data is preprocessed and normalized

print('Shape:' ,sample_X.size())
print('Std:', torch.std_mean(sample_X))
print('Max:', torch.max(sample_X))
print('Min:', torch.min(sample_X))

## Data loading

In [18]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)

In [ ]:
batch_X, batch_y = next(iter(train_loader))
print(batch_X.size())
print(batch_y.size())

In [ ]:
batch_y

## Model

### Logistic regression model

In [21]:
import torch.nn as nn
# Using pytorch nn.Module class
class LogisticRegressionClassifier(nn.Module):
  def __init__(self, input_size, num_classes):
    super().__init__()

    self.linear = nn.Linear(input_size, num_classes)
    self.softmax = nn.Softmax(dim=-1)

  def forward(self, x):
    out = self.linear(x)
    out = self.softmax(out)
    return out

### MLP

In [38]:
class MLP(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super().__init__()
    #TODO
    #hint: use nn.ReLU() layer

  def forward(self,x):
    # TODO
    pass

In [ ]:
# Test the MLP
# net = MLP(input_size=28*28, hidden_size = 100, num_classes=10)
# sample_input = torch.rand(1,784)
# net(sample_input)

## Model creation

In [ ]:
# Pixels on input will be spreaded out
net = LogisticRegressionClassifier(input_size=28*28, num_classes=10)
net

In [23]:
test_input = torch.rand(1, 784)

In [ ]:
net(test_input)

In [ ]:
batch_X, batch_y = next(iter(train_loader))
net(batch_X)

In [ ]:
# Our data shape doesnt match the network input shape
batch_X.size()

In [ ]:
batch_X = batch_X.reshape(-1,28*28)
batch_X.size()

In [ ]:
net(batch_X)

## Training

In [30]:
net = LogisticRegressionClassifier(input_size=784, num_classes=10)

In [40]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

In [41]:
from torchmetrics import Accuracy

accuracy_function = Accuracy(task='multiclass', num_classes=10)

In [ ]:
num_epochs=3
for epoch in range(num_epochs):
  for batch_idx ,(images,labels) in enumerate(train_loader):
    images = images.reshape(-1,784)

    outputs = net(images)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if (batch_idx) % 250 == 0:
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Accuracy: %.4f'
        %(epoch+1, num_epochs, batch_idx, len(train_loader.dataset)//images.size()[0], loss.item(), accuracy_function(outputs,labels)))


In [43]:
from tqdm import tqdm
def get_accuracy(model, loader):
  model.eval()
  all_predictions = []
  all_labels = []
  with torch.no_grad(): #Uses less GPU memory and is faster
    for images,labels in tqdm(loader):
      images = images.reshape(-1,28*28)
      labels = labels

      output = model(images)
      all_predictions.append(output)
      all_labels.append(labels)

  #torch.cat concats tensors along new dimension
  print('Accuracy:', accuracy_function(torch.cat(all_predictions), torch.cat(all_labels)).item())

In [ ]:
get_accuracy(net, train_loader)

## Testing

In [ ]:
test_data = dsets.MNIST(root = './data', train = False, transform = transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size = 128, shuffle = False)

print(test_data.__len__())


In [ ]:
get_accuracy(net, test_loader)

In [ ]:
# Exercise: Solve the problem with MLP

### MLP task solution (Spoiler inside!)

In [ ]:
class MLP(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(input_size, hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size, num_classes),
        nn.Softmax(dim=-1),
    )
  def forward(self,x):
    return self.net(x)